In [4]:
import cv2
import numpy as np
import os
from os.path import isfile, join
import time

tempo = []




#Número de vezes que pretende fazer o processo para avaliar a média de tempo

for teste in range(1):
    inicio = time.time()
    pathA= r'Frames\Frames A\\'
    pathB= r'Frames\Frames B\\'
    pathOut = r'Frames\Frames S\\'
    count=0
    findkey=0
    cortar = 0
    descritor = cv2.xfeatures2d.SIFT_create()
#     descritor = cv2.xfeatures2d.SURF_create()
#     descritor = cv2.BRISK_create()
#     descritor = cv2.ORB_create()
#     matcher =  "Flann" 
    matcher =  "BF"
  

    # Function to extract frames 
    def FrameCapture(path, frame_path,name): 
        pasta=name
        #os.remove(frame_path+pasta)

        # Path to video file 
        vidObj = cv2.VideoCapture(path)

        # Used as counter variable 
        count = 0

        # checks whether frames were extracted 
        success = 1


        while success: 

            # vidObj object calls read 
            # function extract frames 
            success, image = vidObj.read() 


            # Saves the frames with frame-count 
            if image is not None:
                cv2.imwrite(frame_path+name + "frame%d.jpg" % count, image)
                count += 1

    # Driver Code 
    frame_path = r'Frames\\'

    if __name__ == '__main__': 

        # Calling the function 
        FrameCapture(r"Videos\1.mp4", frame_path,"Frames A\\")
        FrameCapture(r"Videos\2.mp4", frame_path,"Frames B\\")
    

#Busca os frames de A e B para fazer a união
    filesA = [f for f in os.listdir(pathA) if isfile(join(pathA, f))]
    filesB = [f for f in os.listdir(pathB) if isfile(join(pathB, f))]
    filesA.sort(key=lambda s: int(''.join(filter(str.isdigit, s))))
    filesB.sort(key=lambda s: int(''.join(filter(str.isdigit, s))))


    if filesA>filesB:
        tamanho=filesB
    else:
        tamanho=filesA
    for i in range(len(tamanho)):#menor quantidade de arquivos entre as duas pastas

        filenameA=filesA[i]
        filenameB=filesB[i]

        if (filenameA and filenameB is not None): 

            img_ = cv2.imread(pathA + filenameA)
            img1 = cv2.cvtColor(img_,cv2.COLOR_BGR2GRAY)
            img = cv2.imread(pathB + filenameB)    
            img2 = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)





            # find key points
            if findkey == 0:

                kp1, des1 = descritor.detectAndCompute(img1,None)
                kp2, des2 = descritor.detectAndCompute(img2,None)

                ##teste FLAN
                if matcher == "flann":
                    FLANN_INDEX_KDTREE = 1
                    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
                    search_params = dict()   # or pass empty dictionary
                    flann = cv2.FlannBasedMatcher(index_params,search_params)
                    matches = flann.knnMatch(des1,des2,k=2)
                else:
                #Teste BFMatcher
                    match = cv2.BFMatcher()
                    matches = match.knnMatch(des1,des2,k=2)
                    

                good = []
                for m,n in matches:
                    if m.distance < 0.5*n.distance:
                        good.append(m)
                findkey = 1    
        #         draw_params = dict(matchColor=(),
        #                                singlePointColor=None,
        #                                flags=2)
        #         img3 = cv2.drawMatches(img_,kp1,img,kp2,good,None,**draw_params)\textit{match}\textit{match}

        #         cv2.imwrite("original_image_drawMatches.jpg", img3)

            MIN_MATCH_COUNT = 10

            if len(good) > MIN_MATCH_COUNT:
                src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5)
                h,w = img1.shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts, M)
                img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    #             cv2.imwrite("original_image_overlapping.jpg", img2)
            count+=1
            #else:
            #    print("Not enought matches are found - %d/%d", (len(good)/MIN_MATCH_COUNT))
            dst = cv2.warpPerspective(img_,M,(img.shape[1] + img_.shape[1], img.shape[0]))
            dst[0:img.shape[0],0:img.shape[1]] = img
            #cortar as imagens de maneira que tire as partes pretas dos cantos
            if cortar == 0:
                rows, cols = np.where(dst[:, :, 0] != 0)
                min_row, max_row = min(rows), max(rows) + 1
                min_col, max_col = min(cols), max(cols) + 1
                cortar =1
            frame = dst [min_row:max_row, min_col:max_col, :]
            final_result=cv2.resize(frame, (640,352))
            cv2.imwrite(pathOut + "frame%d.jpg" % count, final_result)
    #         cv2.imwrite(pathOut + "frame%d.jpg" % count, dst)

        #cv2.imsave("original_image_stitched_crop.jpg", trim(dst))
    # tfim=tinicio-time.time()


    # Este script converte frames para video

    pathInvideo= r'Frames\Frames S\\'
    pathOutvideo = r'Videos\resultado.avi'
    fps = 30

    frame_array = []
    files = [f for f in os.listdir(pathInvideo) if isfile(join(pathInvideo, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    files.sort()

    frame_array = []
    files = [f for f in os.listdir(pathInvideo) if isfile(join(pathInvideo, f)) and f.endswith('jpg')]

    #for sorting the file names properly
    files.sort(key=lambda s: int(''.join(filter(str.isdigit, s))))


    for i in range(len(files)):
        filename=files[i]
        if filename.endswith('.jpg'):
            #reading each files
            img = cv2.imread(pathInvideo + filename)
            if img is not None:
                height, width, layers = img.shape
                size = (width,height)
                #inserting the frames into an image array
                frame_array.append(img)

    out = cv2.VideoWriter(pathOutvideo,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

    fim = time.time()
    resultadoT = round((fim-inicio),3)
    tempo.append(resultadoT)
    #mostra os resultados
    print("Tempo: ",resultadoT," KP1: ", len(kp1)," KP2: ", len(kp2)," Match: ",len(matches),"Good Macth: ", len(good))




Tempo:  40.737  KP1:  4916  KP2:  4348  Match:  4916 Good Macth:  2758
